In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Debugging and Optimizing Agents: A Guide to Tracing in Agent Engine

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/agent-engine/tracing_agents_in_agent_engine.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fagent-engine%2Ftracing_agents_in_agent_engine.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/agent-engine/tracing_agents_in_agent_engine.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/agent-engine/tracing_agents_in_agent_engine.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/agent-engine/tracing_agents_in_agent_engine.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/agent-engine/tracing_agents_in_agent_engine.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/agent-engine/tracing_agents_in_agent_engine.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/agent-engine/tracing_agents_in_agent_engine.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/agent-engine/tracing_agents_in_agent_engine.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>            

| | |
|-|-|
| Author(s) | [Kristopher Overholt](https://github.com/koverholt) |

## Overview

[Agent Engine](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/overview) helps you build and deploy agent-based AI applications that use LLMs and custom tools. Understanding your agent's decision-making process is essential for debugging and optimization, and [Cloud Trace](https://cloud.google.com/trace) is a great tool for exploring this tracing data to get insights.

<img src="https://storage.googleapis.com/github-repo/generative-ai/gemini/agent-engine/images/cloud-trace-agent.png">

This notebook demonstrates how to:

- **Learn Key Concepts**: Learn about the fundamental building blocks of tracing.
- **Deploy Your Agent**: Make your tracing-enabled agent available in a production-like environment on Agent Engine.
- **Enable Tracing**: Enable tracing in a simple agent
- **Examine Traces**: Use the Cloud Console and Cloud Trace SDK to access and analyze a specific trace.

By the end of this notebook, you'll be able to leverage tracing to build more robust and efficient AI agents on Vertex AI.

## Concepts

Here are some of the key concepts and terminology related to tracing, which will be helpful to understand as we explore traces generated by an agent in Agent Engine:

Below is an example of a trace in JSON format, showing a single span. This span represents a call to a large language model (LLM). Notice how the trace data captures important details:

### Example trace

```json
{
   "name": "llm",
   "context": {
       "trace_id": "ed7b336d-e71a-46f0-a334-5f2e87cb6cfc",
       "span_id": "ad67332a-38bd-428e-9f62-538ba2fa90d4"
   },
   "span_kind": "LLM",
   "parent_id": "f89ebb7c-10f6-4bf8-8a74-57324d2556ef",
   "start_time": "2023-09-07T12:54:47.597121-06:00",
   "end_time": "2023-09-07T12:54:49.321811-06:00",
   "status_code": "OK",
   "status_message": "",
   "attributes": {
       "llm.input_messages": [
           {
               "message.role": "system",
               "message.content": "You are an expert Q&A system that is trusted around the world.\nAlways answer the query using the provided context information, and not prior knowledge.\nSome rules to follow:\n1. Never directly reference the given context in your answer.\n2. Avoid statements like 'Based on the context, ...' or 'The context information ...' or anything along those lines."
           },
           {
               "message.role": "user",
               "message.content": "Hello?"
           }
       ],
       "output.value": "assistant: Yes I am here",
       "output.mime_type": "text/plain"
   },
   "events": [],
}
```

### Trace

You can think of a [trace](https://opentelemetry.io/docs/concepts/signals/traces/) like a timeline of requests as they travel through your application. A trace is composed of individual spans, with the first span representing the overall request. Each span provides details about a specific operation within the request.

### Span

A [span](https://opentelemetry.io/docs/concepts/signals/traces/#spans) represents a single unit of work, like a function call or an interaction with an LLM. It captures information such as the operation's name, start and end times, and any relevant attributes (metadata). Spans can be nested, showing parent-child relationships between operations.

### Span Attribute

[Span attributes](https://opentelemetry.io/docs/concepts/signals/traces/#attributes) are key-value pairs that provide additional context about a span. For instance, an LLM span might have attributes like the model name, prompt text, and token count.

### Span Kind

[Span kind](https://opentelemetry.io/docs/concepts/signals/traces/#span-kind) categorizes the type of operation a span represents. Common kinds include:

- `CHAIN`: Links between LLM application steps or the start of a request.
- `LLM`: A call to a large language model.
- `TOOL`: An interaction with an external tool (API, database, etc.).
- `AGENT`: A reasoning block that combines LLM and tool interactions.

## Get started

### Install Vertex AI SDK and other required packages

In [1]:
%pip install --upgrade --user --quiet --force-reinstall \
    "google-cloud-aiplatform[agent_engines,langchain]" \
    cloudpickle==3.0.0 \
    "pydantic>=2.10" \
    google-cloud-trace

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.3.24 requires langsmith<0.4,>=0.1.125, but you have langsmith 0.4.38 which is incompatible.
google-cloud-retail 1.23.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 6.33.0 which is incompatible.
a2a-sdk 0.3.0 requires protobuf==5.29.5, but you have protobuf 6.33.0 which is incompatible.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.3.4 which is incompatible.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.16.0 which is incompatible.
llama-index-legacy 0.9.48.post4 requires tenacity<9.0.0,>=8.2.0, but you have tenacity 9.1.2 which is incompatible.
google-generativeai 0.8.3 requires google-ai-generativelanguage==0.6.10, but you have google-ai-generativelanguage 0.6.18 which is incompatible.
pand

### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

The restart might take a minute or longer. After it's restarted, continue to the next step.

In [2]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [9]:
PROJECT_ID = "argolis-rafaelsanchez-ml-dev"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
STAGING_BUCKET = "gs://argolis-rafaelsanchez-ml-dev"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)

### Authenticate your notebook environment (Colab only)

If you're running this notebook on Google Colab, run the cell below to authenticate your environment.

In [4]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user(project_id=PROJECT_ID)

## Build and deploy an agent

Let's dive into building a simple agent that utilizes tracing. This agent will use a few custom tools to demonstrate how tracing can provide insights into its workflow.

### Import libraries

Before you start building your agent, you'll import the necessary libraries. These include the Vertex AI SDK, pandas for data analysis, and the Cloud Trace SDK for working with trace data.

In [10]:
from datetime import datetime, timedelta

from google.cloud import trace_v1 as trace
import pandas as pd
from vertexai import agent_engines
from vertexai.agent_engines._agent_engines import _utils
from vertexai.preview.reasoning_engines import LangchainAgent

### Define tools

You'll define a few Python functions to act as tools for your agent. These tools will simulate actions or API calls that a real-world agent might perform. For this example, you'll create tools to classify a customer support ticket, query a knowledge base, and escalate a ticket to a human agent.

In [11]:
def classify_ticket(ticket_text: str) -> str:
    """Classifies a support ticket into a category."""
    # Simulate a call to a classification model
    categories = {
        "general": "Questions and information",
        "billing": "Payment and invoices",
        "technical": "API and SDK developer documentation",
    }
    if "payment" in ticket_text:
        category = "billing"
        description = categories[category]
    elif "settings" in ticket_text:
        category = "technical"
        description = categories[category]
    else:
        category = "general"
        description = categories[category]

    return f"This ticket is in the {category} category for questions related to {description}"


def search_knowledge_base(category: str) -> list[dict]:
    """Searches a knowledge base for relevant articles and documentation links."""
    # Simulate a knowledge base search
    articles = {
        "general": [
            {
                "title": "Contacting support",
                "url": "https://example.com/contact",
            }
        ],
        "billing": [
            {
                "title": "How to update your payment information",
                "url": "https://example.com/billing/update",
            },
        ],
        "technical": [
            {
                "title": "Troubleshooting common login issues",
                "url": "https://example.com/technical/help",
            },
        ],
    }
    return articles.get(category, [])


def escalate_to_human(ticket_text: str) -> str:
    """Initiates escalation to a human agent for outage reports."""
    return "Your ticket has been escalated to a human agent. Please expect a response within 1-2 hours."

### Define agent and enable tracing

Now, let's define your agent using the LangChain template in Agent Engine and the Vertex AI SDK. Enable tracing by setting the `enable_tracing` parameter to `True`, which allows you to capture detailed information about the agent's execution.

In [12]:
agent = LangchainAgent(
    model="gemini-2.0-flash",
    model_kwargs={"temperature": 0},
    tools=[classify_ticket, search_knowledge_base, escalate_to_human],
    enable_tracing=True,
)

### Test your agent locally (with traces!)

Let's test your agent locally by sending it a query. Since you've enabled tracing, you'll be able to see how the agent processes this request and interacts with its tools.

In [13]:
agent.query(
    input="""
    Classify the following ticket into a category and give me a relevant documentation link.

    Support ticket text:
    I need to update my billing information since my payment method has expired.
    """
)

Span has more then 32 attributes, some will be truncated
Span has more then 32 attributes, some will be truncated


{'input': '\n    Classify the following ticket into a category and give me a relevant documentation link.\n\n    Support ticket text:\n    I need to update my billing information since my payment method has expired.\n    ',
 'output': 'Okay, I have classified your ticket as being in the "billing" category. Here is a relevant documentation link that should help you update your billing information: https://example.com/billing/update\n'}

### Get your first trace

Before diving deeper into trace analysis, let's use the Cloud Trace SDK to retrieve a specific trace generated by your local agent. This will give you a concrete example to work with.

In [14]:
client = trace.TraceServiceClient()

In [10]:
result = [
    r
    for r in client.list_traces(
        request=trace.types.ListTracesRequest(
            project_id=PROJECT_ID,
            # Return all traces containing `labels {key: "openinference.span.kind" value: "AGENT"}`
            filter="openinference.span.kind:AGENT",
        )
    )
]

In [16]:
trace_data = client.get_trace(project_id=PROJECT_ID, trace_id=result[0].trace_id).spans[
    0
]
trace_data

NameError: name 'result' is not defined

After you deploy your agent and make remote queries in the following sections, you'll dive into the details for working with trace data in the Cloud Console or using the Python SDK for Cloud Trace.

### Deploy your agent

Now that you've seen how tracing works locally, let's deploy your agent to Agent Engine. This will allow you to send it queries in a production-like environment and observe its behavior through traces.

In [17]:
remote_agent = agent_engines.create(
    agent,
    requirements=[
        "google-cloud-aiplatform[agent_engines,langchain]",
        "cloudpickle==3.0.0",
        "pydantic>=2.10",
    ],
)

Identified the following requirements: {'pydantic': '2.12.3', 'cloudpickle': '3.0.0', 'google-cloud-aiplatform': '1.123.0'}
The final list of requirements: ['google-cloud-aiplatform[agent_engines,langchain]', 'cloudpickle==3.0.0', 'pydantic>=2.10']
Using bucket argolis-rafaelsanchez-ml-dev
Wrote to gs://argolis-rafaelsanchez-ml-dev/agent_engine/agent_engine.pkl
Writing to gs://argolis-rafaelsanchez-ml-dev/agent_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://argolis-rafaelsanchez-ml-dev/agent_engine/dependencies.tar.gz
Creating AgentEngine
Create AgentEngine backing LRO: projects/989788194604/locations/us-central1/reasoningEngines/3794128754431754240/operations/8511418140243001344
View progress and logs at https://console.cloud.google.com/logs/query?project=argolis-rafaelsanchez-ml-dev
AgentEngine created. Resource name: projects/989788194604/locations/us-central1/reasoningEngines/3794128754431754240
To use this AgentEngine in another session:
agent

### Query your deployed agent

With your agent deployed, you can interact with it remotely. Let's send a query and generate some trace data to explore.

In [13]:
remote_agent.query(
    input="""
    Classify the following ticket into a category and route the customer accordingly:

    Support ticket text:
    I am unable to make any API calls and I need to report an outage in the system
    """,
)

## Exploring traces in the Cloud Console

The Cloud Trace console provides a powerful and intuitive visual interface for exploring trace data, including visualizing, filtering, and analyzing your traces.

Accessing the Trace Console:

- **Project-Level View**: To see all traces for your Google Cloud project (replace `your-project-id` with your actual project ID), go to: https://console.cloud.google.com/traces/list?project=your-project-id

- **Specific Trace**: If you know the unique Trace ID for a specific trace you want to examine, you can view it directly (replace your-trace-id with the actual Trace ID): https://console.cloud.google.com/traces/list?project=your-project-id&tid=your-trace-id

Features to Explore in the Console:

- **Trace List**: View a list of traces, sorted by start time, along with summary information (duration, number of spans).
- **Waterfall View**: Visualize the spans within a trace as a timeline, showing the duration of each operation and their relationships.
- **Span Details**: Click on a span to view its attributes, including the input and output data, and any custom metadata you've added.
- **Filtering and Search**: The console provides powerful options for filtering traces by time range, service, span name, and other criteria. You can also search for specific traces using keywords or attributes.

For a detailed guide to working with traces in the console, refer to the [Cloud Trace documentation on finding traces](https://cloud.google.com/trace/docs/finding-traces). Experiment with the Cloud Trace console to gain a deeper understanding of your agent's behavior and how it's executing within Agent Engine.

## Working with traces using `pandas`

For more programmatic analysis, you can use the pandas library to work with trace data. You'll fetch traces, convert them to DataFrames, and then use pandas' functionality to explore the trace data.

In [14]:
result = [
    r
    for r in client.list_traces(
        request=trace.types.ListTracesRequest(
            project_id=PROJECT_ID,
            # Return all traces containing `labels {key: "openinference.span.kind" value: "AGENT"}`
            filter="openinference.span.kind:AGENT",
        )
    )
]

In [15]:
trace_data = client.get_trace(project_id=PROJECT_ID, trace_id=result[0].trace_id)

In [16]:
spans = pd.DataFrame.from_records([_utils.to_dict(span) for span in trace_data.spans])
spans.head()

In [17]:
spans[spans["name"] == "ChatVertexAI"]

In [18]:
spans[spans["name"] == "ChatVertexAI"].labels.apply(pd.Series)

## Exploring traces with the Python SDK for Cloud Trace

The Cloud Trace Python SDK provides even more flexibility for working with trace data. We'll use it to demonstrate how to filter traces by date, time, labels, and view types.

**Filter by date and time**

In [19]:
# Calculate the start and end times
now = datetime.utcnow()
yesterday = now - timedelta(hours=24)

# Format the dates as ISO 8601 strings with 'Z' for UTC
end_time = now.isoformat() + "Z"
start_time = yesterday.isoformat() + "Z"

# Request a filtered list of traces by date and time
result = client.list_traces(
    request=trace.types.ListTracesRequest(
        project_id=PROJECT_ID,
        start_time=start_time,
        end_time=end_time,
    )
)

for count, r in enumerate(result):
    if count >= 5:
        break
    print(r)

**Filter by label**

In [20]:
result = client.list_traces(
    request=trace.types.ListTracesRequest(
        project_id=PROJECT_ID,
        # Return traces where any root span's name starts with AgentExecutor
        filter="root:AgentExecutor",
    )
)

for count, r in enumerate(result):
    if count >= 5:
        break
    print(r)

**Filter by view type**

In [21]:
result = client.list_traces(
    request=trace.types.ListTracesRequest(
        project_id=PROJECT_ID,
        # view=trace.types.ListTracesRequest.ViewType.ROOTSPAN,
        view=trace.types.ListTracesRequest.ViewType.MINIMAL,
        # view=trace.types.ListTracesRequest.ViewType.COMPLETE,
    )
)

for count, r in enumerate(result):
    if count >= 5:
        break
    print(r)

## Cleaning up

After you've finished experimenting, it's a good practice to clean up your cloud resources. You can delete the deployed Agent Engine instance to avoid any unexpected charges on your Google Cloud account.

In [22]:
remote_agent.delete()